#### Tensorflow读入数据的方式
1. placeholder声明数据, session.run()的feed参数注入数据  
2. 使用`QueueRunner`导入数据

In [ ]:
with open("file0.csv",'r') as f:
    print('f0 content:\n',f.read())
with open("file1.csv",'r') as f:
    print('f1 content:\n',f.read())

### QueueRunner

#### 一. 基于队列的数据读取步骤

1. The list of filenames
2. Optional filename shuffling
3. Optional epoch limit
4. Filename queue
5. A Reader for the file format
6. A decoder for a record read by the reader
7. Optional preprocessing
8. Example queue

#### 二. queuerunner的标准形式

```python
  #  The list of filenames
  filename_queue = tf.train.string_input_producer([filename], num_epochs=num_epochs)
  # file reader
  reader = tf.TFRecordReader()
  # read row from file
  _, serialized_example = reader.read(filename_queue)
  # parse label and train set
  image,label = decode(serialized_example)
```

In [2]:
import tensorflow as tf
filename_queue = tf.train.string_input_producer(["file0.csv", "file1.csv"])

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# Default values, in case of empty columns. Also specifies the type of the
# decoded result.
record_defaults = [['1'], ['1'], ['1'], ['1'], ['1']]
col1, col2, col3, col4, col5 = tf.decode_csv(value, record_defaults=record_defaults)
features = tf.stack([col1, col2, col3, col4])

with tf.Session() as sess:
    # Start populating the filename queue.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    for i in range(5): # 控制读取的条数, 查过文件总条数的数字会反复读取
        # Retrieve a single instance:
        example,label = sess.run([features, col5])
        print(example)

    coord.request_stop()
    coord.join(threads)

[b"'aa'" b"'bb'" b"'cc'" b"'dd'"]
[b"'aa2'" b"'bb2'" b"'cc2'" b"'dd2'"]
[b"'aa'" b"'bb'" b"'cc'" b"'dd'"]
[b"'aa2'" b"'bb2'" b"'cc2'" b"'dd2'"]
[b"'aa'" b"'bb'" b"'cc'" b"'dd'"]
